<a href="https://colab.research.google.com/github/JulianaCarvajal/Spaceship_Titanic/blob/workOnPreprocess%2FBustamJos3/classConstructionPlayground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working from remote
Upload .py file and import class from it

In [1]:
import experimentalIterator as ei
import pandas as pd #libraries
from zipfile import ZipFile
#call API
%pip install kaggle
#to upload kaggle token .json
from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
files.upload()
#Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json
# Import data files from the Kaggle API
!kaggle competitions download -c spaceship-titanic
#unzip files
file_name = "spaceship-titanic.zip" #the file is the dataset exact name
with ZipFile("spaceship-titanic.zip", 'r') as zip:
  zip.extractall()
  print('Done')
# .zip is now not necessary
!rm *.zip

Saving kaggle.json to kaggle.json
  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 75.2MB/s]
Done


# End of working from remote

# Working from local

In [ ]:
import experimentalIterator as ei
from zipfile import ZipFile
#call API
%pip install kaggle

# End of working from local

# Testing of class

In [15]:
class experimentalIterator: # class construction

    from numpy import concatenate
    global concat
    concat=concatenate
    from sklearn.preprocessing import LabelEncoder #BustamJos3 preprocess  #convert categorical to numerical
    global labelEncoder
    labelEncoder=LabelEncoder()
    from sklearn.model_selection import train_test_split #BustamJos3 for models # import train_test split function
    global train_test
    train_test=train_test_split
    from sklearn.ensemble import RandomForestClassifier #import RandomForest
    global randomForestC
    randomForestC = RandomForestClassifier()
    import seaborn as sns #visualization
    import matplotlib.pyplot as plt
    

    def __init__(self, X, y):
        self._X = X #store as attribute X data of dataset
        self._y = labelEncoder.fit_transform(y.values) #to numeric
        self._preproX=0
        self._XTrain=0
        self._yTrain=0
        self._XTest=0
        self._yTest=0
    
    def dtypeConvertion(self, dataFrame): #convert dtypes to best fit #previous to BustamJos3 preprocess
        df=dataFrame.copy()
        return df.convert_dtypes(infer_objects=True)
    
    def typeSeeker(self, type, dataFrame): #seek for name of cols which match the type
        df=self.dtypeConvertion(dataFrame).copy()
        listDtype=[str(i) for i in (df.dtypes==type).loc[(df.dtypes==type)==True].index] # take name cols which are categorical
        return listDtype

    def convertToNumeric(self, dataFrame):
        df=dataFrame.copy()
        df.drop(['PassengerId','Name'], axis=1,inplace=True)
        listCategoric=self.typeSeeker('string',df)
        df.loc[:,listCategoric].fillna('wanted',inplace=True) #to handle oneHotEncoder, replace NaN values with 'wanted'
        for i in listCategoric: # labelEncoding and replacing consecutively
            array_to_replace=labelEncoder.fit_transform( df.loc[:,i].values ) #labelEncoding
            to_search=len(labelEncoder.classes_)-1 #store No. of categorie for 'wanted
            for j in range(len(array_to_replace)):
                if array_to_replace[j] == to_search:
                    array_to_replace[j]=-1
            df.loc[:,i]=array_to_replace
        return df.loc[:,listCategoric].values

    def preProcess(self):  # start of BustamJos3 preprocess
        XCategoric = self.convertToNumeric(self._X)
        # Transformation on categorical cols with labelEncoder and NaN Imputation categorical
        from sklearn.impute import KNNImputer # nan imputation
        KNNImputer(n_neighbors=1, missing_values=-1,weights='distance', copy=False) #imputation for categoric
        KNNImputer().fit_transform( XCategoric ) #imputation
        from sklearn.preprocessing import OneHotEncoder #oneHotEncoding for categoric cols
        oneHotEncoder=OneHotEncoder(handle_unknown='error', sparse=False)
        convertedOHE=oneHotEncoder.fit_transform(XCategoric[:,[0,2]]) # tranformation to get convertion to OHE  #array with categories of n-categoric cols
        # .reshape on XCategoric for dimensionality  and concatenate
        convertedOHE= concat( (convertedOHE.reshape(-1,8), XCategoric[:,1].reshape(-1,1)), axis=1 )
        # nanImputation for numeric cols
        listNumeric=self.typeSeeker('Int64', self._X) #numeric col names
        XNumeric=self._X[listNumeric].fillna(-1).values # take the numeric types only
        KNNImputer(n_neighbors=1, missing_values=-1,weights='distance')
        imputedNumeric=KNNImputer().fit_transform(XNumeric)
        from sklearn.preprocessing import StandardScaler # standardization
        imputedNumeric=StandardScaler().fit_transform(imputedNumeric) # standardization
        self._preproX = concat( (imputedNumeric, convertedOHE), axis=1 )
        #TODO
        #2. make implementation of selected models according to discussed

    def trainTestSplit(self): #to implement models
        self._XTrain, self._yTrain, self._XTest, self._yTest=train_test(self._preproX, self.y, test_size=0.33, random_state=41)
    
    def gridCVHyperparameter(self, model, params, metric):
        from sklearn.model_selection import GridSearchCV # hyperparameter getter
        gridSearchCV=GridSearchCV(model, params, scoring=metric)
        self.trainTestSplit()
        gridSearchCV.fit(self._XTrain, self._yTrain)
        return model()
    
    def rForestC(self, params, metric):
        bestParams=self.gridCVHyperparameter(randomForestC, params, metric)
        randomForestC()
        return

In [14]:
X, y = pd.read_csv( 'train.csv' ).iloc[:, :-1], pd.read_csv( 'train.csv' ).iloc[:, -1]
exp_iterator = experimentalIterator( X, y )
exp_iterator.preProcess()
exp_iterator._preproX

array([[ 7.21448506e-01, -3.33069630e-01, -2.81012495e-01, ...,
         0.00000000e+00,  1.00000000e+00,  1.49000000e+02],
       [-2.81839266e-01, -1.68040139e-01, -2.75372354e-01, ...,
         0.00000000e+00,  1.00000000e+00,  2.18400000e+03],
       [ 1.99227968e+00, -2.67966252e-01,  1.96000357e+00, ...,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       ...,
       [-1.48067563e-01, -3.33069630e-01, -2.81012495e-01, ...,
         0.00000000e+00,  1.00000000e+00,  5.28500000e+03],
       [ 2.53247546e-01, -3.33069630e-01,  3.76377284e-01, ...,
         0.00000000e+00,  0.00000000e+00,  2.13100000e+03],
       [ 1.05587776e+00, -1.42301594e-01,  2.65687433e+00, ...,
         0.00000000e+00,  1.00000000e+00,  2.13100000e+03]])